In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
import time
import os

In [2]:
nowpath=os.path.abspath('.')
#Commonly used directory：
#/Risk_Detection_Qualification/t_login.csv

In [3]:
def Read_dealt_data(path):
    data=pd.read_csv(nowpath+'/'+path,sep=',')
    data['time_pd']=[pd.Timestamp(chou_large_data.loc[t,'time_pd']) for t in range(len(chou_large_data))]

In [4]:
#Clear the original login data(train，test)
def JD_log_dataclear(df):
    df_cleared=pd.DataFrame(data=df.drop('log_id',1).sort_values(by='time',axis=0))
    df_cleared.index=range(len(df_cleared))
    df_cleared['merge_index']=range(len(df_cleared))
    df_cleared['is_scan_num']=((df_cleared['is_scan']==True)).astype(int)
    df_cleared['is_sec_num']=((df_cleared['is_sec']==True)).astype(int)
    df_cleared['time_pd']=[pd.Timestamp(df_cleared.loc[t,'time']) for t in range(len(df_cleared))]
    df_cleared=df_cleared.drop('is_scan',1)
    df_cleared=df_cleared.drop('is_sec',1)
    df_cleared=df_cleared.drop('time',1)
    return df_cleared

In [5]:
#Clear the original trade data(train，test)
def JD_trade_dataclear(df):
    df_cleared=pd.DataFrame(data=df.sort_values(by='time',axis=0))
    df_cleared.index=range(len(df_cleared))
    df_cleared['time_pd']=[pd.Timestamp(df_cleared.loc[t,'time']) for t in range(len(df_cleared))]
    df_cleared=df_cleared.drop('time',1)
    df_cleared['weekdays']=[df_cleared.loc[t,'time_pd'].weekday() for t in range(len(df_cleared))]
    df_cleared['hour'] = df_cleared['time_pd'].dt.hour
    return df_cleared

In [6]:
#Split data with time stamp(train，test)
def JD_train_test_split(indata,time_split):
    out_train=indata[indata.loc[:,'time_pd']<time_split]
    out_test=indata[indata.loc[:,'time_pd']>time_split]
    return (out_train,out_test)

In [7]:
#Calculate counting relationship between two cols, usable with login and trade(train，test)
def JD_corr_columns(df,col1,col2): #count kinds of col2 for each col1
    name_corr='{}_per_{}'.format(col2,col1)
    name_c1='record_for_this_{}'.format(col1)
    df_cd=df.sort_values(by=[col1,col2],ascending=[True,True])
    df_cd[name_corr]=np.nan
    df_cd[name_c1]=np.nan
    df_cd.index=range(len(df_cd))
    for i in range(len(df_cd)):
        if np.isnan(df_cd.loc[i,name_corr]):
            thislen=len(df_cd[df_cd.loc[:,col1]==df_cd.loc[i,col1]])
            #print(thislen)
            df_cd.loc[df_cd.loc[:,col1]==df_cd.loc[i,col1],name_c1]=thislen
            df_cd.loc[df_cd.loc[:,col1]==df_cd.loc[i,col1],name_corr]=len(df_cd.loc[df_cd.loc[:,col1]==df_cd.loc[i,col1],col2].value_counts())
        if i%1000==0:
            print(i)
    return df_cd

In [8]:
#Calculate proba of every unique value in col2 appears in a particular cal1 element
#, and record if this value is different with the last and next.
#Ex: col1 is id, col2 is ip
def JD_rate_consistency_element(df,col1,col2,timeline):
    name_rate='{}_appear_rate_per_{}'.format(col2,col1)
    name_diff_former='{}_diff_with_this_{}_last_login'.format(col2,col1)
    name_diff_later='{}_diff_with_this_{}_next_login'.format(col2,col1)
    df_cd=df.sort_values(by=[col1,timeline],ascending=[True,True])
    df_cd[name_rate]=np.nan
    df_cd[name_diff_former]=np.nan
    df_cd[name_diff_later]=np.nan
    df_cd.index=range(len(df_cd))
    for i in range(len(df_cd)):
        if np.isnan(df_cd.loc[i,name_rate]):
            thislen=len(df_cd[df_cd.loc[:,col1]==df_cd.loc[i,col1]])
            #The first should be seen "same" with last, and last "same" with next
            df_cd.loc[df_cd[df_cd.loc[:,col1]==df_cd.loc[i,col1]].index[0],name_diff_former]=0
            df_cd.loc[df_cd[df_cd.loc[:,col1]==df_cd.loc[i,col1]].index[-1],name_diff_later]=0
            idx=df_cd[df_cd.loc[:,col1]==df_cd.loc[i,col1]].index
            for j in idx:
                #print('j is:',j)
                #time3=time.time()
                col2len=len(df_cd[df_cd.loc[:,col1]==df_cd.loc[i,col1]]\
                            [df_cd[df_cd.loc[:,col1]==df_cd.loc[i,col1]].loc[:,col2]==df_cd.loc[j,col2]])
                if np.isnan(df_cd.loc[j,name_rate]):
                    #record proba of this col2 value for this cal1 element.
                    df_cd.loc[j,name_rate]=col2len/thislen
                if 'j_last' in dir():
                    #time5=time.time()
                    if df_cd.loc[j,col2]==df_cd.loc[j_last,col2]:
                        df_cd.loc[j,name_diff_former]=0
                        df_cd.loc[j_last,name_diff_later]=0
                    else:
                        df_cd.loc[j,name_diff_former]=1
                        df_cd.loc[j_last,name_diff_later]=1 
                j_last=j
            del j_last
        if i%1000==0:
            print(i)
    return(df_cd)

In [9]:
def login_timedelta(df,col1,timeline): #count timedelta for col1 record(compared with last and next)
    name_c1='timedelta_{}_with_last'.format(col1)
    name_c2='timedelta_{}_with_next'.format(col1)
    df_cd=df.sort_values(by=[col1,timeline],ascending=[True,True])
    df_cd[name_c1]=np.nan
    df_cd[name_c2]=np.nan
    df_cd.index=range(len(df_cd))
    for i in range(len(df_cd)):
        if np.isnan(df_cd.loc[i,name_c1]):
            #The first should be seen "same" with last, and last "same" with next
            df_cd.loc[df_cd[df_cd.loc[:,col1]==df_cd.loc[i,col1]].index[0],name_c1]=50
            df_cd.loc[df_cd[df_cd.loc[:,col1]==df_cd.loc[i,col1]].index[-1],name_c2]=50
            idx=df_cd[df_cd.loc[:,col1]==df_cd.loc[i,col1]].index
            for j in idx:
                if 'j_last' in dir():
                    df_cd.loc[j,name_c1]=(df_cd.loc[j,timeline]-df_cd.loc[j_last,timeline]).seconds//3600
                    df_cd.loc[j_last,name_c2]=df_cd.loc[j,name_c1]
                j_last=j
            del j_last
        if i%1000==0:
            print(i)
    return df_cd

In [ ]:
df_log=pd.read_csv(nowpath+'/df_log_Dec10.csv',sep=',')
df_log['time_pd']=[pd.Timestamp(df_log.loc[t,'time_pd']) for t in range(len(df_log))]
#df_log['merge_index']=range(len(df_log))
df_log=login_timedelta(df_log,'id','time_pd')
df_log=login_timedelta(df_log,'device','time_pd')
#df_log.to_csv('/df_log_Dec10_eve.csv',sep=',')

In [ ]:
#df_log['time_pd']=[pd.Timestamp(df_log.loc[t,'time_pd']) for t in range(len(df_log))]
# df_log['merge_index']=range(len(df_log))
df_log=login_timedelta(df_log,'id','time_pd')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [ ]:
df_log.to_csv('df_log_Dec11_1.csv',sep=',')
df_log=login_timedelta(df_log,'device','time_pd')
df_log.to_csv('df_log_Dec11_2.csv',sep=',')

In [10]:
#Calculate risk proba. Merged data(train, test)
def JD_risk_rate(df,colrisk,col2): #count rate of risk for col2
    name_c1='record_for_{}'.format(col2)
    name_rate='risk_rate_on_{}'.format(col2)
    df_cd=df.sort_values(by=col2,ascending=True)
    df_cd[name_c1]=np.nan
    df_cd[name_rate]=np.nan
    df_cd.index=range(len(df_cd))
    for i in range(len(df_cd)):
        tmp=df_cd[df_cd.loc[:,col2]==df_cd.loc[i,col2]]
        if np.isnan(df_cd.loc[i,name_rate]):
            thislen=len(tmp[tmp.loc[:,col2]==df_cd.loc[i,col2]])
            df_cd.loc[df_cd.loc[:,col2]==df_cd.loc[i,col2],name_c1]=thislen
            df_cd.loc[df_cd.loc[:,col2]==df_cd.loc[i,col2],name_rate]=tmp.loc[:,colrisk].mean()
    return(df_cd)

In [11]:
#Record and drop rows whose col2 value is in list_result. Times means sum of all counters, based on col1. Ex: col1 is id，col2 isresult
def JD_useless_result_count(df,col1,col2,list_result):
    name_c1='Times_loginfail_for_this_{}'.format(col1)

    df_cd=df.sort_values(by=[col1,col2],ascending=[True,True])

    df_cd[name_c1]=np.nan
    df_cd.index=range(len(df_cd))
    for i in range(len(df_cd)):
        if np.isnan(df_cd.loc[i,name_c1]):
            tmp=df_cd[df_cd.loc[:,col1]==df_cd.loc[i,col1]]
            counter=0
            for result in list_result:
                counter=counter+len(tmp[tmp.loc[:,col2]==result])
            df_cd.loc[df_cd.loc[:,col1]==df_cd.loc[i,col1],name_c1]=counter
        if i%1000==0:
            print(i)
#        for result in list_result:
#        df_cd=df_cd[df_cd.loc[:,'result']!=result]

    return(df_cd)

In [12]:
#Create dummy for each element in list_result
def JD_result_dummy(df,col,list_result):
    for result in list_result:
        name_c1='result_{}_for_this_record'.format(result)
        df[name_c1]=0
        df.loc[df.loc[:,col]==result,name_c1]=1
    return(df)

In [13]:
#If a row with a col1 element has col2 value in list_result, mark as 1.
def JD_result_mark(df,col1,col2,list_result):
    df_cd=df
    name_c1='ID_appear_result_{}'.format(list_result)
    df_cd=df_cd.sort_values(by=[col1,col2],ascending=[True,True])
    df_cd[name_c1]=np.nan
    df_cd.index=range(len(df_cd))
    for i in range(len(df_cd)):
        if np.isnan(df_cd.loc[i,name_c1]):
            tmp=df_cd[df_cd.loc[:,col1]==df_cd.loc[i,col1]]
            df_cd.loc[df_cd.loc[:,col1]==df_cd.loc[i,col1],name_c1]=int(bool(len(tmp[tmp.loc[:,col2]==list_result])))
        #df_cd=df_cd.drop(tmp.loc[:,'result'] in list_result)
        if i%1000==0:
            print(i)
    return(df_cd)

In [14]:
#merge login data and trade. If drop_late_login==True, then drop login data later than trade data,
# and drop those rows which could not be matched with id.
def JD_Merge_Train(df_log,df_trade_use,drop_late_login=False,merge_test=False): 
    df_trade_use['rowkey_of_log']=0
    df_log_use=df_log[['id','time_pd','merge_index']]
    #df_trade_use['ip_per_id']=0
    #df_trade_use=df_trade_use.drop(('device','log_from'),1)
    #this_row='Null'
    nofit_count=0
    nofit_danger_count=0
    trade_before_log=0
    df_log_use=df_log_use.sort_values(by=['id','time_pd'],axis=0,ascending=[False,False])
    df_trade_use=df_trade_use.sort_values(by=['id','time_pd'],axis=0,ascending=[False,True])
    #df_log_use=df_log_use.sort(['id','time_pd'],ascending=[True,False])
    df_log_use.index=range(len(df_log_use))
    df_trade_use.index=range(len(df_trade_use))
    #df_log_use['index_new']=range(len(df_log_use))
    #print(df_log_use['index_new'])
    for i in range(len(df_trade_use)):
        df_tmp=df_log_use[df_log_use.loc[:,'id']==df_trade_use.loc[i,'id']]
        sigsig=False
        #row='Null'
        #this_row='Null'
        for rowd in df_tmp.iterrows():
            row=rowd[1]
            #if df_trade_use.loc[i,'time_pd']-row['time_pd']<pd.Timedelta('01:00:00'):
            if df_trade_use.loc[i,'time_pd']>row['time_pd']:
                df_trade_use.loc[i,'rowkey_of_log']=row["merge_index"]
                sigsig=True

                del row
                break
            #this_row=row.copy()
            #print(this_row)
        if sigsig==False:
            if drop_late_login==False:
                if 'row' in dir():
                    df_trade_use.loc[i,'rowkey_of_log']=row["merge_index"]
                    trade_before_log=trade_before_log+1
                    del row
                else:
                    nofit_count=nofit_count+1
                    if merge_test==False:
                        if df_trade_use.loc[i,'is_risk']==1:
                            nofit_danger_count=nofit_danger_count+1
                    #trade_none=trade_none.append(df_trade_use.loc[i])
                    df_trade_use=df_trade_use.drop(i,axis=0)
            else:
                df_trade_use=df_trade_use.drop(i,axis=0)
        if i%1000==0 and i >0:
            print(i)

    try:
        df_trade_use=df_trade_use.merge(df_log,left_on='rowkey_of_log',right_on='merge_index',how='inner')
        df_trade_use['time_dif']=df_trade_use['time_pd_x']-df_trade_use['time_pd_y']
        #df_trade_use=df_trade_use.drop('Unnamed: 0',axis=1).drop('merge_index',axis=1)
        df_trade_use=df_trade_use.drop('Unnamed: 0',axis=1)
    except:
        print("Couldn't merge the cols!")
    print(nofit_count)
    print(trade_before_log)
    return df_trade_use

In [21]:
list_useless=[31,6,-1,3,-4]
list_useful=[-2,22]
list_5=5
list_23=23
# df_log=pd.read_csv(nowpath+'/Risk_Detection_Qualification/t_login_test.csv',sep=',')
df_trade=pd.read_csv(nowpath+'/Risk_Detection_Qualification/t_trade.csv',sep=',')
# df_trade_1_6=pd.read_csv(nowpath+'/Risk_Detection_Qualification/t_trade.csv',sep=',')
# df_log_use=JD_log_dataclear(df_log)
df_trade_use=JD_trade_dataclear(df_trade)
# df_trade_1_6_use=JD_trade_dataclear(df_trade_1_6)
# df_log_deal=JD_corr_columns(df_log_use,'id','ip')
# df_trade_1_6_deal=JD_risk_rate(df_trade_1_6_use,'is_risk','id')
df_log_deal=JD_useless_result_count(df_cd_log,'id','result',list_useless)
df_log_deal=JD_result_dummy(df_log_deal,'result',list_useful)
df_log_deal=JD_result_mark(df_log_deal,'id','result',list_5)
df_log_deal=JD_result_mark(df_log_deal,'id','result',list_23)
df_log_deal.to_csv('log_1_7_use_Dec7.csv',sep=',')


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


519000
520000
521000
522000
523000
524000
525000
526000
527000
528000
529000
530000
531000
532000
533000
534000
535000
536000
537000
538000
539000
540000
541000
542000
543000
544000
545000
546000
547000
548000
549000
550000
551000
552000
553000
554000
555000
556000
557000
558000
559000
560000
561000
562000
563000
564000
565000
566000
567000
568000
569000
570000
571000
572000
573000
574000
575000
576000
577000
578000
579000
580000
581000
582000
583000
584000
585000
586000
587000
588000
589000
590000
591000
592000
593000
594000
595000
596000
597000
598000
599000
600000
601000
602000
603000
604000
605000
606000
607000
608000
609000
610000
611000
612000
613000
614000
615000
616000
617000
618000
619000
620000
621000
622000
623000
624000
625000
626000
627000
628000
629000
630000
631000
632000
633000
634000
635000
636000
637000
638000
639000
640000
641000
642000
643000
644000
645000
646000
647000
648000
649000
650000
651000
652000
653000
654000
655000
656000
657000
658000
659000
660000
661000

KeyError: "['merge_index'] not in index"

In [3]:
df_log=pd.read_csv(nowpath+'/df_log_Dec10.csv',sep=',')
# df_log['time_pd']=[pd.Timestamp(df_log.loc[t,'time_pd']) for t in range(len(df_log))]
# #df_log['merge_index']=range(len(df_log))
# df_log=login_timedelta(df_log,'id','time_pd')
# df_log=login_timedelta(df_log,'device','time_pd')
df_log

,timelong,device,log_from,ip,city,result,timestamp,type,id,is_scan_num,...,result_-2_for_this_record,result_22_for_this_record,ID_appear_result_23,merge_index,city_appear_rate_per_id,city_diff_with_this_id_last_login,city_diff_with_this_id_next_login,device_appear_rate_per_id,device_diff_with_this_id_last_login,device_diff_with_this_id_next_login
0,17797,786238,1,505101,400,1,1430000000,3,5,0,...,0,0,0,2,0.500000,0,0,0.250000,0,1
1,6000,781684,2,505101,400,1,1430000000,3,5,0,...,0,0,0,1,0.500000,0,1,0.250000,1,1
2,37972,77727,1,1300209,398,1,1430000000,3,5,0,...,0,0,0,0,0.500000,1,0,0.500000,1,0
3,31864,77727,1,1300209,398,31,1430000000,3,5,0,...,0,0,0,3,0.500000,0,0,0.500000,0,0
4,6000,620460,2,888204,238,6,1420000000,3,6,0,...,0,0,0,8,1.000000,0,0,0.444444,0,0
5,8000,620460,2,888204,238,6,1420000000,3,6,0,...,0,0,0,9,1.000000,0,0,0.444444,0,0
6,8000,620460,2,888204,238,1,1420000000,3,6,0,...,0,0,0,6,1.000000,0,0,0.444444,0,1
7,8000,703894,2,1289501,238,1,1420000000,3,6,0,...,0,0,0,7,1.000000,0,0,0.111111,1,1
8,10000,68936,2,899325,238,31,1430000000,3,6,0,...,0,0,0,10,1.000000,0,0,0.444444,1,0
9,8000,68936,2,899325,238,31,1430000000,3,6,0,...,0,0,0,11,1.000000,0,0,0.444444,0,0


In [5]:
df_log['time_pd']=[pd.Timestamp(df_log.loc[t,'time_pd']) for t in range(len(df_log))]
# df_log['merge_index']=range(len(df_log))
df_log=login_timedelta(df_log,'id','time_pd')

In [45]:
df_log=df_log.drop('Unnamed: 0',axis=1).drop('Unnamed: 0.1',axis=1).drop('Unnamed: 0.1.1',axis=1)

In [ ]:
df_log.to_csv('df_log_Dec10_night.csv',sep=',')

In [ ]:
df_log=pd.read_csv(nowpath+'/df_log_Dec11_1.csv',sep=',')
df_log['time_pd']=[pd.Timestamp(df_log.loc[t,'time_pd']) for t in range(len(df_log))]
df_trade=pd.read_csv(nowpath+'/Risk_Detection_Qualification/t_trade.csv',sep=',')
df_trade_use=JD_trade_dataclear(df_trade)
df_merged=JD_Merge_Train(df_log,df_trade_use,merge_test=False)
df_merged.to_csv('train_use_Dec11.csv',sep=',')

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000


In [15]:
df_log=pd.read_csv(nowpath+'/df_log_Dec11_1.csv',sep=',')
df_log['time_pd']=[pd.Timestamp(df_log.loc[t,'time_pd']) for t in range(len(df_log))]
df_trade_test=pd.read_csv(nowpath+'/Risk_Detection_Qualification/t_trade_test.csv',sep=',')
df_trade_test_use=JD_trade_dataclear(df_trade_test)
df_merged_test=JD_Merge_Train(df_log,df_trade_test_use,merge_test=True)
df_merged_test.to_csv('test_use_Dec11.csv',sep=',')

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
0
123


In [50]:
df_merged_test

,rowkey,id_x,time_pd_x,weekdays,hour,rowkey_of_log,timelong,device,log_from,ip,...,result_22_for_this_record,ID_appear_result_23,merge_index,city_appear_rate_per_id,city_diff_with_this_id_last_login,city_diff_with_this_id_next_login,device_appear_rate_per_id,device_diff_with_this_id_last_login,device_diff_with_this_id_next_login,time_dif
0,24622,175943,2015-07-05 19:30:59,6,19,683077,35000.0,346531,2,541982,...,0,0.0,683077,1.000000,0.0,0.0,0.250000,1.0,0.0,1 days 05:50:16
1,39036,175943,2015-07-08 14:27:34,2,14,683077,35000.0,346531,2,541982,...,0,0.0,683077,1.000000,0.0,0.0,0.250000,1.0,0.0,4 days 00:46:51
2,62674,175932,2015-07-12 23:08:38,6,23,682975,6000.0,584682,2,874961,...,0,0.0,682975,0.066667,1.0,1.0,0.066667,1.0,1.0,0 days 02:14:11
3,163053,175932,2015-07-29 18:54:44,2,18,682983,162611.0,656035,1,742444,...,0,0.0,682983,0.266667,1.0,0.0,0.066667,1.0,0.0,13 days 21:53:54
4,50616,175903,2015-07-10 18:58:38,4,18,682933,11669.0,730637,1,795051,...,0,0.0,682933,0.844828,0.0,0.0,0.568966,0.0,0.0,0 days 03:45:21
5,68355,175889,2015-07-13 23:25:34,0,23,682866,6000.0,660658,2,1103212,...,0,0.0,682866,1.000000,0.0,0.0,0.500000,0.0,1.0,0 days 00:03:59
6,23239,175873,2015-07-05 14:50:41,6,14,682704,15178.0,696532,1,1335219,...,0,0.0,682704,0.978261,0.0,0.0,0.304348,0.0,0.0,0 days 00:19:25
7,94869,175873,2015-07-18 16:01:32,5,16,682705,18990.0,696532,1,1335219,...,0,0.0,682705,0.978261,0.0,0.0,0.304348,0.0,0.0,0 days 00:32:11
8,79351,175798,2015-07-16 03:32:28,3,3,682584,52097.0,684993,1,1263340,...,0,0.0,682584,0.225000,0.0,1.0,0.025000,1.0,1.0,0 days 00:04:08
9,81268,175798,2015-07-16 13:11:18,3,13,682572,6000.0,377722,2,451870,...,0,0.0,682572,0.575000,0.0,1.0,0.375000,0.0,1.0,0 days 00:48:15
